In [4]:
# Loading required libraries 
from langchain_core.prompts import PromptTemplate
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [3]:
# extarct the data from PDF
def load_pdf(data_dir):
    loader =DirectoryLoader(data_dir,
                            glob = "*.pdf",
                            loader_cls =PyPDFLoader)
    documents = loader.load()
    return documents

In [13]:
extracted_data = load_pdf(r"C:\github\openai\basic_assignment\data")

In [14]:
extracted_data

[Document(page_content='THERE once lived a bir d and her two new-bor n babies in a\nforest. They had a nest in a tall, shady tree and there the\nmother bird took care of her little ones day and night.\nOne day, ther e was a big stor m. Ther e was thunder ,\nlightning and rain, and the wind blew down many trees. The\ntall tree in which the birds lived also came down. A big, heavy\nbranch hit the nest and killed the bird. Fortunately for the\nbaby birds, the strong wind blew them away to the other side\nof the forest. One of them came down near a cave where a\ngang of robbers lived. The other landed outside a rishi’s\nashram a little distance away.•A mother bird and her two young ones lived in a forest.\n•The mother was killed in a stor m and the young bir ds wer e\nseparated fr om each other .\n•Each found a dif ferent home.1 11111A Tale of Two\nBirds\nRationalised 2023-24\n', metadata={'source': 'C:\\github\\openai\\basic_assignment\\data\\fepw101.pdf', 'page': 0}),
 Document(page_cont

In [15]:
len(extracted_data)

40

In [19]:
# creating textn chunks 
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 300,
        chunk_overlap = 30
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [20]:
text_chunks = text_split(extracted_data)

In [21]:
len(text_chunks)

190

In [5]:
embeddings = OpenAIEmbeddings()

c:\Users\Aditya\anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [25]:
# storinng dataset into vector database
db = FAISS.from_documents(documents= text_chunks,embedding=embeddings)

In [26]:
db.save_local("faiss_index")



In [6]:
db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)

In [31]:
query = " How did the two baby birds get separated?"
docs = db.similarity_search(query)

In [32]:
docs

[Document(page_content='what he has always heard. He welcomes people to the ashram.\nNow, come inside and rest. I’ll tell you more about this place\nand these birds.”\nQuestions\n1.How did the two baby birds get separated?\n2.Where did each of them find a home?\n3.What did the first bird say to the stranger?', metadata={'source': 'C:\\github\\openai\\basic_assignment\\data\\fepw101.pdf', 'page': 2}),
 Document(page_content='gang of robbers lived. The other landed outside a rishi’s\nashram a little distance away.•A mother bird and her two young ones lived in a forest.\n•The mother was killed in a stor m and the young bir ds wer e\nseparated fr om each other .\n•Each found a dif ferent home.1 11111A Tale of Two\nBirds', metadata={'source': 'C:\\github\\openai\\basic_assignment\\data\\fepw101.pdf', 'page': 0}),
 Document(page_content='THERE once lived a bir d and her two new-bor n babies in a\nforest. They had a nest in a tall, shady tree and there the\nmother bird took care of her little

In [7]:
from langchain_community.chat_models import ChatOpenAI

In [21]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 


In [9]:
llm = ChatOpenAI(model="gpt-3.5-turbo")

c:\Users\Aditya\anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [10]:
prompt = PromptTemplate.from_template(
    """
    Answer the following question based on the provided context.
    Think step by step before providing a detailed answer.
    I will tip you $1000 if the user finds the answer helpful.
    <context> 
    {context}
    </context>
    question:
    {input}
    """
)

In [11]:
from langchain.chains.combine_documents import create_stuff_documents_chain

In [12]:
document_chain = create_stuff_documents_chain(llm,prompt)

In [13]:
retriver=db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [14]:
retriver

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000027FF9369A80>, search_kwargs={'k': 3})

In [15]:
from langchain.chains import create_retrieval_chain

In [16]:
retrival_chain = create_retrieval_chain(retriver,document_chain)

In [17]:
response = retrival_chain.invoke({"input":"what is dream?"})


In [18]:
print(response["answer"])

Based on the provided context, a dream is described as an activity of the mind that occurs when we are asleep. Dreams can be both probable and improbable, with some potentially reflecting aspects of one's problems but not capable of predicting the future. Additionally, dreams are noted for their ability to help individuals sleep through noise and disturbances.


In [19]:
def retrieval_qa_chain(user_input):
    document_chain = create_stuff_documents_chain(llm,prompt)
    retriver=db.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    retrival_chain = create_retrieval_chain(retriver,document_chain)
    response = retrival_chain.invoke({"input":user_input})
    return response["answer"]


In [20]:
retrieval_qa_chain("what is dream?")

"Based on the provided context, a dream is defined as an activity of the mind that occurs when we are asleep. It is a phenomenon where the mind creates scenarios and experiences that may be probable or improbable. Dreams are important in helping us sleep through noise and other disturbances, and they may also reveal something about a person's problems. However, dreams cannot predict the future."